In [ ]:
# 数据根目录
data_path = "/home/queahren/文档/A28_Data_Test"

In [ ]:
# ResNet 提取坐标轴文本数据，并生成基准数据
from resNet.getNum import getNum
getNum(data_path)

In [ ]:
# 预处理数据，生成灰度图于 gray 文件夹、unet预测的兴趣区域 mask 图于 mask 文件夹

import cv2
import os
from PIL import Image
from matplotlib import pyplot as plt
import shutil

filepath = data_path + "/images"
filename = os.listdir(filepath)  #图像名列表
base_dir = filepath + "/"
new_dir = data_path + "/mask/" #以\\结尾
gray_dir = data_path + "/gray/"

if not os.path.exists(new_dir):
    os.mkdir(new_dir)
if not os.path.exists(gray_dir):
    os.mkdir(gray_dir)
    
for img in filename:
    if(img.endswith('.png')):
        im = cv2.imread(base_dir+img)      
        gray_img = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        ret, binary_img = cv2.threshold(gray_img, -1, 255, cv2.THRESH_BINARY)       
        cv2.imwrite(new_dir+img.split('.')[0] + "_mask.png", binary_img)
        cv2.imwrite(gray_dir+img, gray_img)

In [ ]:
# UNET 提取折线/曲线 至 result 文件夹
 
from unet.predict_unet import predict_unet
import cv2

save_path = data_path + "/result/"

base_dir = data_path + "/images"
filename = os.listdir(base_dir)
base_dir = data_path + "/"

if not os.path.exists(save_path):
    os.mkdir(save_path)

count = 0
for imgname in filename:
    if(imgname.endswith('.png')):
        count += 1
        print(imgname, count/len(filename))
        predict_unet(base_dir, imgname, save_path)

In [ ]:
# 通过转化为 HSV 空间，利用 UNet 获取到的 result 进行进一步提取，置于 enhanceResult 文件夹中 

from enhanceMask import enhanceMask
data_path = "/home/queahren/文档/A28_Data_Test"
enhanceMask(data_path,"line")
enhanceMask(data_path,"curve")


In [ ]:
# 合并提取结果至 mergeResult 文件夹
import shutil
import distutils.dir_util
import os

merge_path = data_path + "/mergeResult/"

   
distutils.dir_util.copy_tree(data_path + '/result',merge_path) 
distutils.dir_util.copy_tree(data_path + '/enhanceResult',merge_path) 

In [ ]:
# 依据基准数据 与 分割result 进行采点并预估对应 Y 值， 输出结果于 check_test 及 根目录下xlsx文件
from createYpixel import createYpixel
from createExcel import createExcel
data_path = "/home/queahren/文档/A28_Data_Test"
createYpixel(data_path)
createExcel(data_path)